<a href="https://colab.research.google.com/github/HayfaLK/Projects/blob/master/LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi
!pip install kaleido
!pip install python-multipart
!pip install uvicorn
!pip install cohere tiktoken
!pip install llmx
!pip install --upgrade llmx
!pip install openpyxl
!pip install --upgrade pandas
!pip install openai
!pip install replicate
import pandas as pd
import os
import random
import replicate
import json
import re
import csv
import time
import numpy as np
import pylab
from google.colab import drive


In [ ]:
from google.colab import files
uploaded = files.upload()
# Extraire le nom du fichier
file_name = list(uploaded.keys())[0]

In [ ]:
def split_words_on_uppercase(input_string):
    result = []
    current_word = ""

    for char in input_string:
        # Check if the character is uppercase
        if char.isupper():
            # Add the current word to the result list
            result.append(current_word)
            # Start a new word with the uppercase character
            current_word = char
        else:
            # Continue building the current word
            current_word += char

    # Add the last word to the result list
    result.append(current_word)

    # Join the words with a space and return the result
    return ' '.join(result).strip()  # Remove leading and trailing spaces

In [ ]:
import chardet
import pandas as pd

# Function to detect encoding
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']
# Get the detected encoding
detected_encoding = detect_encoding(file_name)

In [ ]:
#file = 'AwardToArtist.csv'
file_name = os.path.basename(file_name)
print (file_name)
file_name_without_extension = os.path.splitext(file_name)[0]


cleaned2 = file_name_without_extension.split("To")

partie1 = cleaned2[0]
partie2 = cleaned2[1]

print(partie1)
print(partie2)


# Example usage:
#print(partie1)
#print(partie2)
input_string = partie1
partie11 = split_words_on_uppercase(input_string)
print(partie11)
input_string = partie2
partie22 = split_words_on_uppercase(input_string)
print(partie22)


db = pd.read_csv(file_name, engine='python',encoding = detected_encoding, on_bad_lines='skip')
current_columns = db.columns

# Renommez les colonnes en utilisant leurs indices
db.columns = [partie1, partie2]

# Affichez le DataFrame mis à jour
print(db)

# Remove rows with empty elements
db = db.dropna()
db = db.drop_duplicates()
# Replace or remove non-alphanumeric characters in a specific column
db[partie1] = db[partie1].str.replace(r'[^a-zA-Z0-9\s.]', '', regex=True)
db[partie2] = db[partie2].str.replace(r'[^a-zA-Z0-9\s.]', '', regex=True)
# Save the cleaned DataFrame back to a new CSV file
cleaned_file_name = f"cleaned{file_name}"
db.to_csv(cleaned_file_name, index=False)
db

In [ ]:
from replicate.client import Client

replicate = Client(api_token="...")

In [ ]:
#os.environ["REPLICATE_API_TOKEN"] = ".."

In [ ]:
#token = "..."

**TEXT FORMAT**

In [ ]:
import re

# Créez une variable pour stocker les noms de fichiers
similarity_files = []
json_result_list = []

# Create an empty list to store all JSON items
all_json_items = []

liste_examples = []
liste_answer = []
# Nombre total d'itérations (adaptez-le à votre cas)
nombre_iterations = 3
for i in range(nombre_iterations):
  #all_ceos_xml = ET.Element('all_ceos')
  # Mesurer le temps d'exécution initial
  temps_debut = time.time()
  print(f"*****************************Iteration {i+1}*********************************")
  premiere_iteration = True

  # Convertissez le DataFrame en une liste à chaque itération
  ma_liste = db.values.tolist()
  elements_choisis = random.sample(ma_liste, 5)

  #print(f"Éléments choisis: {elements_choisis}")
  for j in range(len(elements_choisis)):
    Example_j = elements_choisis[j][0]
    Answer_j = elements_choisis[j][1]
    liste_examples.append(Example_j)
    liste_answer.append(Answer_j)
    #print(f"Example_{j}: {Example_j}")




# Supprimez les éléments choisis de la liste principale
  for element in elements_choisis:
      ma_liste.remove(element)

  #print(f"Reste de la liste: {ma_liste}")

  batch_size = 8
  total_rows = len(ma_liste)
  num_batches = (total_rows + batch_size - 1) // batch_size  # Calculating the number of batches

  for batch_index in range(num_batches):
    #print(f"Batch:",{batch_index+1})
    start_index = batch_index * batch_size
    end_index = min((batch_index + 1) * batch_size, total_rows)
    batch_list = ma_liste[start_index:end_index]
    #print(f"Batch List: {batch_list}")

    # prendre la premiere colonne de la liste "ma_liste" et le mettre dans "output_list"
    output_list = []
    for row in batch_list:
        output_list.append(row[0])

    # Cette ligne de code transforme une liste d'éléments en une chaîne de caractères où les éléments sont séparés par des virgules et un espace.
    formatted_output = ', '.join(output_list)



    input_data = {
          "prompt": f"""Look at this examples: \n\
        Example: {liste_examples[0]} \n\
        Answer: {liste_answer[0]}\n\
        Example: {liste_examples[1]} \n\
        Answer: {liste_answer[1]}\n\
        Example: {liste_examples[2]} \n\
        Answer: {liste_answer[2]}\n\
        Example: {liste_examples[3]} \n\
        Answer: {liste_answer[3]}\n\
        Example: {liste_examples[4]} \n\
        Answer: {liste_answer[4]}\n\
        I order you to focus carefully on my prompt before answering. \n\
        complete the {partie22} of each one of this list: {formatted_output} \n\
        you should provide me just {len(batch_list)} rows. \n\
        If you don't know the answer please write 'unknown' \n\
        If more than one answer fit for {partie22} write only the first option""",
        "system_prompt": fr"""You are a semantic data transformer.\n\
        Provide the answer in this format: \n\
        {partie11}: \n\
        {partie22}: \n\
        Do not write explanations. Do not type questions unless I instruct you to do so."""
      }

    print(f"Prompt: {input_data}")

    output_prompt2 = replicate.run(
        "replicate/llama-2-70b-chat:58d078176e02c219e11eb4da5a02a7830a283b14cf8f94537af893ccff5ee781", input=input_data)
    full_response_prompt2 = "".join(output_prompt2)

    print(full_response_prompt2)
    # Extract answers from the response using regex
    matches = re.findall(rf"{partie11}: (.+?)\n{partie22}: (.+?)\n", full_response_prompt2)
    # Extend data_list with matches from this iteration
    address_list = []
    address_list.extend(matches)


    # Create a CSV file to save the JSON data for this iteration
    csv_file_name = f"output_{i+1}.csv"

    # Déterminez le mode d'ouverture en fonction de l'itération et premiere_iteration
    if premiere_iteration:
      mode = 'w'
    else:
      mode = 'a'


    with open(csv_file_name, mode=mode, newline='') as csv_file:
        # Créer un écrivain CSV
        csv_writer = csv.writer(csv_file)
        if premiere_iteration:
          # Écrire l'en-tête dans le fichier CSV
          csv_writer.writerow([partie1, partie2])

        # Write data
        for data in address_list:
          address = data[0]
          state_world = data[1]
          csv_writer.writerow([address, state_world])

    # Imprimer un message indiquant que les données CSV ont été sauvegardées
    print(f'Données CSV ont été sauvegardées dans {csv_file_name}')
    premiere_iteration = False
  # Assuming you have defined 'df' and 'csv_file_name' variables earlier in your code
  df2 = pd.read_csv(f'output_{i+1}.csv')
  # Créez une liste vide pour stocker les lignes de df3
  result_rows = []

  # Parcourez les lignes de df
  for index, row_df in db.iterrows():
    airport_df = row_df[partie1]
    country_df = row_df[partie2]

    # Recherchez si la ligne existe dans df2 en fonction de la colonne "airport"
    matching_row_df2 = df2[df2[partie1] == airport_df]

    if not matching_row_df2.empty:
      airport_df2 = matching_row_df2[partie1].iloc[0]
      country_df2 = matching_row_df2[partie2].iloc[0]

      # Calculez la similarité entre les éléments correspondants (vous pouvez personnaliser cette logique)
      similarity = 1 if country_df == country_df2 else 0

      # Ajoutez la ligne à la liste
      result_rows.append([country_df, country_df2, similarity])

  # Créez un DataFrame df3 à partir de la liste
  df3 = pd.DataFrame(result_rows, columns=[ 'data_from_df', 'data_from_df2', 'similarity'])

  # Affichez df3
  print(df3)

  # Enregistrer le DataFrame df3 dans un fichier CSV
  df3.to_csv(f'Similarity_{i+1}.csv', index=False)

  # Print a message indicating that the data has been saved
  print(f'Data with similarity values has been saved to \'Similarity_{i+1}.csv\'')

  # Mesurer le temps d'exécution final
  temps_fin = time.time()
  temps_execution = temps_fin - temps_debut
  # Convertir le temps en minutes et secondes
  minutes = int(temps_execution // 60)
  secondes = int(temps_execution % 60)
  # Formater le temps en chaîne de caractères
  temps_formate = f"{minutes} min {secondes} s"

  # Enregistrez le temps d'exécution formate dans un fichier texte
  with open(f'temps_execution_{i+1}.txt', 'w') as fichier_temps:
    fichier_temps.write(f'Temps d\'exécution total : {temps_formate}')

  # Affichez le temps d'exécution formate
  print(f"Temps d'exécution total : {temps_formate}")


**XML FORMAT**

# Fonction extraire xml

In [ ]:
import re

def extract_text_between_tags(text, tag_name):
    pattern = f"<{tag_name}>((?:(.|\n)+?)?)</{tag_name}>"
    match = re.search(pattern, text)
    if match:
        return match.group(0)
    else:
        raise ValueError(f"No {tag_name} tag found in the text")

In [ ]:
import xml.etree.ElementTree as ET
import re

# Créez une variable pour stocker les noms de fichiers
similarity_files = []

# Create an empty list to store all JSON items
all_json_items = []

liste_examples = []
liste_answer = []
# Nombre total d'itérations (adaptez-le à votre cas)
nombre_iterations = 3
for i in range(nombre_iterations):
  #all_ceos_xml = ET.Element('all_ceos')
  # Mesurer le temps d'exécution initial
  temps_debut = time.time()
  print(f"*****************************Iteration {i+1}*********************************")
  premiere_iteration = True

  # Convertissez le DataFrame en une liste à chaque itération
  ma_liste = db.values.tolist()
  elements_choisis = random.sample(ma_liste, 5)

  #print(f"Éléments choisis: {elements_choisis}")
  for j in range(len(elements_choisis)):
    Example_j = elements_choisis[j][0]
    Answer_j = elements_choisis[j][1]
    liste_examples.append(Example_j)
    liste_answer.append(Answer_j)
    #print(f"Example_{j}: {Example_j}")




# Supprimez les éléments choisis de la liste principale
  for element in elements_choisis:
      ma_liste.remove(element)

  #print(f"Reste de la liste: {ma_liste}")

  batch_size = 8
  total_rows = len(ma_liste)
  num_batches = (total_rows + batch_size - 1) // batch_size  # Calculating the number of batches

  for batch_index in range(num_batches):
    #print(f"Batch:",{batch_index+1})
    start_index = batch_index * batch_size
    end_index = min((batch_index + 1) * batch_size, total_rows)
    batch_list = ma_liste[start_index:end_index]
    #print(f"Batch List: {batch_list}")

    # prendre la premiere colonne de la liste "ma_liste" et le mettre dans "output_list"
    output_list = []
    for row in batch_list:
        output_list.append(row[0])

    # Cette ligne de code transforme une liste d'éléments en une chaîne de caractères où les éléments sont séparés par des virgules et un espace.
    formatted_output = ', '.join(output_list)



    input_data = {
          "prompt": f"Act as Senior data analyst, \n\
        Look at this examples: \n\
        Example: {liste_examples[0]} \n\
        Answer: {liste_answer[0]}\n\
        Example: {liste_examples[1]} \n\
        Answer: {liste_answer[1]}\n\
        Example: {liste_examples[2]} \n\
        Answer: {liste_answer[2]}\n\
        Example: {liste_examples[3]} \n\
        Answer: {liste_answer[3]}\n\
        Example: {liste_examples[4]} \n\
        Answer: {liste_answer[4]}\n\
        I order you to focus carefully on my prompt before answering: \n\
        complete the {partie22} of each one of this list: {formatted_output} \n\
        Provide the result in string format, that begining with 'TEXTBEGIN' and ending with 'TEXTEND'.\n\
        you should provide me just {len(batch_list)} rows. \n\
        If you don't know the answer please write 'unknown' \n\
        If more than one answer fit for {partie22} write only the first option"


      }

    #print(f"Prompt: {input_data}")

    output_prompt2 = replicate.run(
        "replicate/llama-2-70b-chat:58d078176e02c219e11eb4da5a02a7830a283b14cf8f94537af893ccff5ee781", input=input_data)
    full_response_prompt2 = "".join(output_prompt2)

    print( full_response_prompt2)
    TEXT = extract_text_between_tags(full_response_prompt2, "query")
    #print("TEXXXXXXXXXXXT",TEXT)
    # Analyser le XML
    root = ET.fromstring(TEXT)
    # Create a CSV file to save the JSON data for this iteration
    csv_file_name = f"output_{i+1}.csv"

    # Déterminez le mode d'ouverture en fonction de l'itération et premiere_iteration
    if premiere_iteration:
      mode = 'w'
    else:
      mode = 'a'


    with open(csv_file_name, mode=mode, newline='') as csv_file:
        # Créer un écrivain CSV
        csv_writer = csv.writer(csv_file)
        if premiere_iteration:
          # Écrire l'en-tête dans le fichier CSV
          csv_writer.writerow([partie1, partie2])

        # Parcourir chaque élément 'ceo' dans le XML
        for ceo_element in root.findall('.//row'):
            # Extraire les valeurs d'adresse et de State_World
            address = ceo_element.find(partie1).text
            state_world = ceo_element.find(partie2).text

            # Écrire les valeurs dans le fichier CSV
            csv_writer.writerow([address, state_world])

    # Imprimer un message indiquant que les données CSV ont été sauvegardées
    print(f'Données CSV ont été sauvegardées dans {csv_file_name}')
    premiere_iteration = False
  # Assuming you have defined 'df' and 'csv_file_name' variables earlier in your code
  df2 = pd.read_csv(f'output_{i+1}.csv')
  # Créez une liste vide pour stocker les lignes de df3
  result_rows = []

  # Parcourez les lignes de df
  for index, row_df in db.iterrows():
    airport_df = row_df[partie1]
    country_df = row_df[partie2]

    # Recherchez si la ligne existe dans df2 en fonction de la colonne "airport"
    matching_row_df2 = df2[df2[partie1] == airport_df]

    if not matching_row_df2.empty:
      airport_df2 = matching_row_df2[partie1].iloc[0]
      country_df2 = matching_row_df2[partie2].iloc[0]

      # Calculez la similarité entre les éléments correspondants (vous pouvez personnaliser cette logique)
      similarity = 1 if country_df == country_df2 else 0

      # Ajoutez la ligne à la liste
      result_rows.append([country_df, country_df2, similarity])

  # Créez un DataFrame df3 à partir de la liste
  df3 = pd.DataFrame(result_rows, columns=[ 'data_from_df', 'data_from_df2', 'similarity'])

  # Affichez df3
  print(df3)

  # Enregistrer le DataFrame df3 dans un fichier CSV
  df3.to_csv(f'Similarity_{i+1}.csv', index=False)

  # Print a message indicating that the data has been saved
  print(f'Data with similarity values has been saved to \'Similarity_{i+1}.csv\'')

  # Mesurer le temps d'exécution final
  temps_fin = time.time()
  temps_execution = temps_fin - temps_debut
  # Convertir le temps en minutes et secondes
  minutes = int(temps_execution // 60)
  secondes = int(temps_execution % 60)
  # Formater le temps en chaîne de caractères
  temps_formate = f"{minutes} min {secondes} s"

  # Enregistrez le temps d'exécution formate dans un fichier texte
  with open(f'temps_execution_{i+1}.txt', 'w') as fichier_temps:
    fichier_temps.write(f'Temps d\'exécution total : {temps_formate}')

  # Affichez le temps d'exécution formate
  print(f"Temps d'exécution total : {temps_formate}")

**JSON FORMAT**

**0 shot prompt**

In [ ]:
# Créez une variable pour stocker les noms de fichiers
similarity_files = []

# Create an empty list to store all JSON items
all_json_items = []

liste_examples = []
liste_answer = []
# Nombre total d'itérations (adaptez-le à votre cas)
nombre_iterations = 3
for i in range(nombre_iterations):
  # Mesurer le temps d'exécution initial
  temps_debut = time.time()
  print(f"*****************************Iteration {i+1}*********************************")
  premiere_iteration = True

  # Convertissez le DataFrame en une liste à chaque itération
  ma_liste = db.values.tolist()
  elements_choisis = random.sample(ma_liste, 0)

  print(f"Éléments choisis: {elements_choisis}")
  for j in range(len(elements_choisis)):
    Example_j = elements_choisis[j][0]
    Answer_j = elements_choisis[j][1]
    liste_examples.append(Example_j)
    liste_answer.append(Answer_j)
    #print(f"Example_{j}: {Example_j}")




# Supprimez les éléments choisis de la liste principale
  for element in elements_choisis:
      ma_liste.remove(element)

  print(f"Reste de la liste: {ma_liste}")

  batch_size = 8
  total_rows = len(ma_liste)
  num_batches = (total_rows + batch_size - 1) // batch_size  # Calculating the number of batches

  for batch_index in range(num_batches):
    print(f"Batch:",{batch_index+1})
    start_index = batch_index * batch_size
    end_index = min((batch_index + 1) * batch_size, total_rows)
    batch_list = ma_liste[start_index:end_index]
    print(f"Batch List: {batch_list}")

    # prendre la premiere colonne de la liste "ma_liste" et le mettre dans "output_list"
    output_list = []
    for row in batch_list:
        output_list.append(row[0])

    # Cette ligne de code transforme une liste d'éléments en une chaîne de caractères où les éléments sont séparés par des virgules et un espace.
    formatted_output = ', '.join(output_list)

    # Construisez le JSON avec des noms de PDG correctement formatés (avec des espaces)
    ceos_json = []
    for ceo_name in output_list:
        ceos_json.append({partie1: ceo_name, partie2: 'valueee'})  # Remplacez 'value2' par la valeur appropriée

    input_data = {
        "prompt": f"Act as Senior data analyst, \n\
        I order you to focus carefully on my prompt before answering: \n\
        complete the {partie2} of each one of this list: {formatted_output} \n\
        Provide the result in JSON string format.\n\
        and this is the format: {json.dumps(ceos_json)}\n\
        you should provide me just {len(batch_list)} rows. \n\
        If you don't know the answer please write 'unknown' \n\
        If more than one answer fit for {partie2} write only the first option"
      }

    print(f"Prompt: {input_data}")

    output_prompt2 = replicate.run(
        "replicate/llama-2-70b-chat:58d078176e02c219e11eb4da5a02a7830a283b14cf8f94537af893ccff5ee781", input=input_data)
    full_response_prompt2 = "".join(output_prompt2)

    print(f"Replicate: {full_response_prompt2}")

    json_result_list = []
    inside_json = False
    json_string = ""
    added_items = set()

    for line in full_response_prompt2:
        if line.strip().startswith('{'):
            inside_json = True
            json_string = line
        elif inside_json:
            json_string += line
        if line.endswith('}'):
            inside_json = False
            try:
                json_obj = json.loads(json_string)
                json_str = json.dumps(json_obj)  # Convertir le dictionnaire en une chaîne JSON
                if json_str not in added_items:  # Utiliser la chaîne JSON comme clé
                    json_result_list.append(json_obj)
                    print(json_result_list)
                    added_items.add(json_str)
            except json.JSONDecodeError:
                pass

    # Add the JSON items from this batch to the list of all JSON items
    all_json_items.extend(json_result_list)

    #print("all_json_items: ",all_json_items)

    # Create a CSV file to save the JSON data for this iteration
    csv_file_name = f"output_{i+1}.csv"

    # Déterminez le mode d'ouverture en fonction de l'itération et premiere_iteration
    if premiere_iteration:
      mode = 'w'
    else:
      mode = 'a'

    # Utilisez le mode déterminé pour ouvrir le fichier CSV
    with open(csv_file_name, mode=mode, newline='') as csv_file:
      fieldnames = [partie1, partie2]  # Définir les noms de champs pour le CSV
      writer = csv.writer(csv_file)
      # Si c'est la première itération, écrivez l'en-tête
      if premiere_iteration:
        writer.writerow(fieldnames)  # Écrire l'en-tête

      # Write all JSON items to the CSV with removed double quotes
      for json_item in json_result_list:
        company = json_item[partie1].strip('"')
        ceo = json_item[partie2].strip('"')  # Remove quotes around CEO
        writer.writerow([company, ceo])


    # Print a message indicating that the data has been saved for this iteration
    print(f"JSON data for iteration {i+1} has been saved to {csv_file_name}")

    premiere_iteration = False
  # Assuming you have defined 'df' and 'csv_file_name' variables earlier in your code
  df2 = pd.read_csv(f'output_{i+1}.csv')
  # Créez une liste vide pour stocker les lignes de df3
  result_rows = []

  # Parcourez les lignes de df
  for index, row_df in db.iterrows():
    airport_df = row_df[partie1]
    country_df = row_df[partie2]

    # Recherchez si la ligne existe dans df2 en fonction de la colonne "airport"
    matching_row_df2 = df2[df2[partie1] == airport_df]

    if not matching_row_df2.empty:
      airport_df2 = matching_row_df2[partie1].iloc[0]
      country_df2 = matching_row_df2[partie2].iloc[0]

      # Calculez la similarité entre les éléments correspondants (vous pouvez personnaliser cette logique)
      similarity = 1 if country_df == country_df2 else 0

      # Ajoutez la ligne à la liste
      result_rows.append([country_df, country_df2, similarity])

  # Créez un DataFrame df3 à partir de la liste
  df3 = pd.DataFrame(result_rows, columns=[ 'data_from_df', 'data_from_df2', 'similarity'])

  # Affichez df3
  print(df3)

  # Enregistrer le DataFrame df3 dans un fichier CSV
  df3.to_csv(f'Similarity_{i+1}.csv', index=False)

  # Print a message indicating that the data has been saved
  print(f'Data with similarity values has been saved to \'Similarity_{i+1}.csv\'')

  # Mesurer le temps d'exécution final
  temps_fin = time.time()
  temps_execution = temps_fin - temps_debut
  # Convertir le temps en minutes et secondes
  minutes = int(temps_execution // 60)
  secondes = int(temps_execution % 60)
  # Formater le temps en chaîne de caractères
  temps_formate = f"{minutes} min {secondes} s"

  # Enregistrez le temps d'exécution formate dans un fichier texte
  with open(f'temps_execution_{i+1}.txt', 'w') as fichier_temps:
    fichier_temps.write(f'Temps d\'exécution total : {temps_formate}')

  # Affichez le temps d'exécution formate
  print(f"Temps d'exécution total : {temps_formate}")


In [ ]:
pip install --upgrade replicate


In [ ]:
# Créez une variable pour stocker les noms de fichiers
similarity_files = []

# Create an empty list to store all JSON items
all_json_items = []

liste_examples = []
liste_answer = []
# Nombre total d'itérations (adaptez-le à votre cas)
nombre_iterations = 3
for i in range(nombre_iterations):
  # Mesurer le temps d'exécution initial
  temps_debut = time.time()
  print(f"*****************************Iteration {i+1}*********************************")
  premiere_iteration = True

  # Convertissez le DataFrame en une liste à chaque itération
  ma_liste = db.values.tolist()
  elements_choisis = random.sample(ma_liste, 3)

  print(f"Éléments choisis: {elements_choisis}")
  for j in range(len(elements_choisis)):
    Example_j = elements_choisis[j][0]
    Answer_j = elements_choisis[j][1]
    liste_examples.append(Example_j)
    liste_answer.append(Answer_j)
    #print(f"Example_{j}: {Example_j}")




# Supprimez les éléments choisis de la liste principale
  for element in elements_choisis:
      ma_liste.remove(element)

  print(f"Reste de la liste: {ma_liste}")

  batch_size = 8
  total_rows = len(ma_liste)
  num_batches = (total_rows + batch_size - 1) // batch_size  # Calculating the number of batches

  for batch_index in range(num_batches):
    print(f"Batch:",{batch_index+1})
    start_index = batch_index * batch_size
    end_index = min((batch_index + 1) * batch_size, total_rows)
    batch_list = ma_liste[start_index:end_index]
    print(f"Batch List: {batch_list}")

    # prendre la premiere colonne de la liste "ma_liste" et le mettre dans "output_list"
    output_list = []
    for row in batch_list:
        output_list.append(row[0])

    # Cette ligne de code transforme une liste d'éléments en une chaîne de caractères où les éléments sont séparés par des virgules et un espace.
    formatted_output = ', '.join(output_list)

    # Construisez le JSON avec des noms de PDG correctement formatés (avec des espaces)
    ceos_json = []
    for ceo_name in output_list:
        ceos_json.append({partie1: ceo_name, partie2: 'valueee'})  # Remplacez 'value2' par la valeur appropriée



    input_data = {
        "prompt": f"Act as Senior data analyst, \n\
        Look at this examples: \n\
        Example: {liste_examples[0]} \n\
        Answer: {liste_answer[0]}\n\
        Example: {liste_examples[1]} \n\
        Answer: {liste_answer[1]}\n\
        Example: {liste_examples[2]} \n\
        Answer: {liste_answer[2]}\n\
        I order you to focus carefully on my prompt before answering: \n\
        complete the {partie2} of each one of this list: {formatted_output} \n\
        Provide the result in JSON string format.\n\
        and this is the format: {json.dumps(ceos_json)}\n\
        you should provide me just {len(batch_list)} rows. \n\
        If you don't know the answer please write 'unknown' \n\
        If more than one answer fit for {partie2} write only the first option"
      }

    print(f"Prompt: {input_data}")

    output_prompt2 = replicate.run(
        "replicate/llama-2-70b-chat:58d078176e02c219e11eb4da5a02a7830a283b14cf8f94537af893ccff5ee781", input=input_data)
    full_response_prompt2 = "".join(output_prompt2)

    print(f"Replicate: {full_response_prompt2}")

    json_result_list = []
    inside_json = False
    json_string = ""
    added_items = set()

    for line in full_response_prompt2:
        if line.strip().startswith('{'):
            inside_json = True
            json_string = line
        elif inside_json:
            json_string += line
        if line.endswith('}'):
            inside_json = False
            try:
                json_obj = json.loads(json_string)
                json_str = json.dumps(json_obj)  # Convertir le dictionnaire en une chaîne JSON
                if json_str not in added_items:  # Utiliser la chaîne JSON comme clé
                    json_result_list.append(json_obj)
                    print(json_result_list)
                    added_items.add(json_str)
            except json.JSONDecodeError:
                pass

    # Add the JSON items from this batch to the list of all JSON items
    all_json_items.extend(json_result_list)

    #print("all_json_items: ",all_json_items)

    # Create a CSV file to save the JSON data for this iteration
    csv_file_name = f"output_{i+1}.csv"

    # Déterminez le mode d'ouverture en fonction de l'itération et premiere_iteration
    if premiere_iteration:
      mode = 'w'
    else:
      mode = 'a'

    # Utilisez le mode déterminé pour ouvrir le fichier CSV
    with open(csv_file_name, mode=mode, newline='') as csv_file:
      fieldnames = [partie1, partie2]  # Définir les noms de champs pour le CSV
      writer = csv.writer(csv_file)
      # Si c'est la première itération, écrivez l'en-tête
      if premiere_iteration:
        writer.writerow(fieldnames)  # Écrire l'en-tête

      # Write all JSON items to the CSV with removed double quotes
      for json_item in json_result_list:
        company = json_item[partie1].strip('"')
        ceo = json_item[partie2].strip('"')  # Remove quotes around CEO
        writer.writerow([company, ceo])


    # Print a message indicating that the data has been saved for this iteration
    print(f"JSON data for iteration {i+1} has been saved to {csv_file_name}")

    premiere_iteration = False
  # Assuming you have defined 'df' and 'csv_file_name' variables earlier in your code
  df2 = pd.read_csv(f'output_{i+1}.csv')
  # Créez une liste vide pour stocker les lignes de df3
  result_rows = []

  # Parcourez les lignes de df
  for index, row_df in db.iterrows():
    airport_df = row_df[partie1]
    country_df = row_df[partie2]

    # Recherchez si la ligne existe dans df2 en fonction de la colonne "airport"
    matching_row_df2 = df2[df2[partie1] == airport_df]

    if not matching_row_df2.empty:
      airport_df2 = matching_row_df2[partie1].iloc[0]
      country_df2 = matching_row_df2[partie2].iloc[0]

      # Calculez la similarité entre les éléments correspondants (vous pouvez personnaliser cette logique)
      similarity = 1 if country_df == country_df2 else 0

      # Ajoutez la ligne à la liste
      result_rows.append([country_df, country_df2, similarity])

  # Créez un DataFrame df3 à partir de la liste
  df3 = pd.DataFrame(result_rows, columns=[ 'data_from_df', 'data_from_df2', 'similarity'])

  # Affichez df3
  print(df3)

  # Enregistrer le DataFrame df3 dans un fichier CSV
  df3.to_csv(f'Similarity_{i+1}.csv', index=False)

  # Print a message indicating that the data has been saved
  print(f'Data with similarity values has been saved to \'Similarity_{i+1}.csv\'')

  # Mesurer le temps d'exécution final
  temps_fin = time.time()
  temps_execution = temps_fin - temps_debut
  # Convertir le temps en minutes et secondes
  minutes = int(temps_execution // 60)
  secondes = int(temps_execution % 60)
  # Formater le temps en chaîne de caractères
  temps_formate = f"{minutes} min {secondes} s"

  # Enregistrez le temps d'exécution formate dans un fichier texte
  with open(f'temps_execution_{i+1}.txt', 'w') as fichier_temps:
    fichier_temps.write(f'Temps d\'exécution total : {temps_formate}')

  # Affichez le temps d'exécution formate
  print(f"Temps d'exécution total : {temps_formate}")


In [ ]:
# Liste pour stocker les résultats individuels
results = []
# Chaîne pour stocker les résultats sous forme de texte
result_str = ""
# Dictionnaire pour stocker les résultats
resultats = {}
# Parcourez les fichiers pour chaque itération
for i in range(3):
  # Chargez le fichier "Similarity_airportToCountry_{i+1}.csv" en tant que DataFrame pandas
  nom_fichier = f'Similarity_{i+1}.csv'
  similarity_file = pd.read_csv(nom_fichier)
  # Calculez la somme de la troisième colonne
  sum_of_column_3 = similarity_file['similarity'].sum()
  # Obtenez le nombre de lignes
  num_rows = len(similarity_file)
  result = sum_of_column_3 / num_rows
  result_str += f"The accuracy of {nom_fichier} : {result}\n"
  results.append(result)  # Ajoutez le résultat à la liste

# Calculez la moyenne des résultats
average_result = np.average(results)
result_str += f"The average : {average_result}\n"
ecart_type = np.std(results)
result_str += f"The standard deviation : {ecart_type}\n"

# Enregistrez les résultats dans un fichier texte
with open("resultat.txt", "w") as file:
  file.write(result_str)
  print("Résultats enregistrés dans 'resultat.txt'")

Résultats enregistrés dans 'resultat.txt'


**++++++++++++++++++++TEST+++++++++++++++++++++**

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

# Two lists of words
word1 = ['North Rhine-Westphalia']
word2 = ['North Rhine-Westphalia, Germany']



#Compute embedding for both lists
embeddings1 = model.encode(word1, convert_to_tensor=True)
embeddings2 = model.encode(word2, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(word1)):
        print("{} \t\t {} \t\t Score: {:.4f}".format(word1[i], word2[i], cosine_scores[i][i]))

North Rhine-Westphalia 		 North Rhine-Westphalia, Germany 		 Score: 0.9664


In [ ]:
pip install -U sentence-transformers


In [ ]:
import nltk
nltk.edit_distance("North Rhine-Westphalia", "North Rhine-Westphalia, Germany")

9